# Custom Similarity/Distance function

In [6]:
def similarity_c_custom(C1,C2,dist_fun):
    U1, s1, _ = np.linalg.svd(C1, hermitian=True, full_matrices=False)
    U2, s2, _ = np.linalg.svd(C2, hermitian=True, full_matrices=False)
    
    if dist_fun == "eucl":
        d = 0
        
        U1 = U1 @ np.diag(s1)
        U2 = U2 @ np.diag(s2)
        for i1, row1 in enumerate(U1):
            for i2, row2 in enumerate(U2):
                aligned_row2 = (1 - 2 * (row1 @ row2 < 0)) * row2
                d += linalg.norm(row1 - aligned_row2)

        return d / (linalg.norm(s1)+linalg.norm(s2))
    
    elif dist_fun == "cos":
        return linalg.norm(
            np.diag(s1)**0.5 @ U1.T @ U2 @ np.diag(s2)**0.5)**2 / (linalg.norm(s1)*linalg.norm(s2)
        )

dist_fun = 'eucl'
a = np.array([[1,1,1],[2,2,2],[2,2,2]])
b = np.array([[1,1,1],[2,0,2],[2,2,2]])
d = similarity_c_custom(a,a,dist_fun)
print(d)
d = similarity_c_custom(a,b,dist_fun)
print(d)
#d = similarity_c_custom(Cs[phonemes.index("f")],Cs[phonemes.index("h#")],dist_fun)
#print(d)

0.34815531191139504
0.8924878041920855


# Overlaps after clustering

In [50]:
overlaps = np.zeros((len(phonemes), len(phonemes))) # (Cs x phonemes)

for C_idx, assignment in enumerate(assignments):
    for idx in assignment:
        overlaps[C_idx, phonemes.index(labs[idx])] += 1

dic = {}

for _ in range(len(phonemes)):
    C_idx, phoneme_idx = np.unravel_index(overlaps.argmax(), overlaps.shape)
    dic[phonemes[phoneme_idx]] = C_idx
    overlaps[C_idx,:] = np.zeros((1,len(phonemes)))
    overlaps[:,phoneme_idx] = np.zeros((len(phonemes),))

Cs_final = []
Ns_final = []
for phoneme in [p for p in phonemes if p in list(dic.keys()) ]:
    Cs_final.append(Cs_kmeans[dic[phoneme]])
    Ns_final.append(Ns_kmeans[dic[phoneme]])